# 🧠 NeuroTrader Ultimate: Fast Boot Edition ⚡

This version uses **Google Drive Caching** to speed up setup from **5 minutes -> 30 seconds**.
It also prevents "Version Conflicts" because it freezes the working environment!

In [ ]:
#@title ⚙️ Training Configuration
#@markdown Select the level you want to train (1=MLP, 2=LSTM)
TRAINING_LEVEL = 2 #@param {type:"integer"}

## 1. Init & Connect Drive ☁️

In [ ]:
import os
import sys
import time
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# Config
CACHE_DIR = "/content/drive/MyDrive/NeuroTrader_Workspace/cache"
ENV_TAR = os.path.join(CACHE_DIR, "neurotrader_env_v1.tar.gz")
EXTRACT_PATH = "/content/neuro_env"

# Create Workspace Dirs if not exist
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs("/content/drive/MyDrive/NeuroTrader_Workspace/logs", exist_ok=True)
os.makedirs("/content/drive/MyDrive/NeuroTrader_Workspace/models", exist_ok=True)

## 2. Fast Environment Setup 🚀
Automatically detects if a cached environment exists. If not, creates one.

In [ ]:
def setup_environment():
    start_time = time.time()
    
    # A. Check if Cache Exists
    if os.path.exists(ENV_TAR):
        print(f"📦 Found cached environment: {ENV_TAR}")
        print("⏳ Extracting... (This takes about 30-45s)")
        
        # Extract to /content
        !tar -xzf "$ENV_TAR" -C /content
        
        # Add to Path
        site_packages = os.path.join(EXTRACT_PATH, "lib/python3.10/site-packages")
        sys.path.insert(0, site_packages)
        
        print(f"✅ Environment Loaded in {time.time() - start_time:.1f}s!")
        
    # B. First Time Setup (Slow)
    else:
        print("⚠️ No cache found. Starting Fresh Install (This takes ~5 mins)... ")
        
        # 1. Clone Repo for requirements
        if not os.path.exists("NeuroTrader"):
             !git clone https://github.com/MaDoHee33/NeuroTrader.git
        
        # 2. Create Custom Env Dir
        !python -m venv $EXTRACT_PATH
        
        # 3. Activate & Install
        # We use the pip inside the new venv
        PIP_CMD = f"{EXTRACT_PATH}/bin/pip"
        
        !$PIP_CMD install --upgrade pip
        # Force install clean dependencies
        !$PIP_CMD install -r NeuroTrader/requirements_colab.txt
        
        # 4. Pack it up!
        print("💾 Caching environment to Drive... (Do not close tab)")
        !tar -czf "$ENV_TAR" -C /content neuro_env
        
        # 5. Add to current session path
        site_packages = os.path.join(EXTRACT_PATH, "lib/python3.10/site-packages")
        sys.path.insert(0, site_packages)
        
        print(f"✨ Setup & Cache Complete in {time.time() - start_time:.1f}s!")

setup_environment()

## 3. Clone Repository 🐙
Pull latest code.

In [ ]:
REPO_URL = "https://github.com/MaDoHee33/NeuroTrader.git"

if not os.path.exists("NeuroTrader"):
    !git clone $REPO_URL
else:
    %cd NeuroTrader
    !git pull
    %cd ..

## 4. Run Training 🏃‍♂️

In [ ]:
# Ensure we use the python interpreter from our custom env
PYTHON_EXEC = f"{EXTRACT_PATH}/bin/python"

%cd /content/NeuroTrader

print(f"🚀 Starting Training for Level {TRAINING_LEVEL}...")
# Run script using the custom environment python
!$PYTHON_EXEC src/brain/train_ppo.py --level $TRAINING_LEVEL

In [ ]:
%load_ext tensorboard
# View Logs for Current Level
LOG_SUBDIR = f"L{TRAINING_LEVEL}_MLP" if TRAINING_LEVEL == 1 else f"L{TRAINING_LEVEL}_LSTM"
LOG_PATH = f"/content/drive/MyDrive/NeuroTrader_Workspace/logs/{LOG_SUBDIR}"

print(f"📊 Opening TensorBoard for: {LOG_PATH}")
%tensorboard --logdir "$LOG_PATH"